## Sixth Session (Related to the Course Project)

---------------

## Graph Classification with [Deep Graph Library (DGL)](https://docs.dgl.ai/index.html) for the graduate course "[Graph Machine learning](https://github.com/zahta/graph_ml)"

### Dataset: BACE

##### by [Zahra Taheri](https://github.com/zahta), 06 June 2023

---------------

### This Tutorial Is Prepared Based on the Following References

- [FunQG: Molecular Representation Learning via Quotient Graphs](https://pubs.acs.org/doi/10.1021/acs.jcim.3c00445)
- [Supporting Information of FunQG](https://pubs.acs.org/doi/suppl/10.1021/acs.jcim.3c00445/suppl_file/ci3c00445_si_001.pdf)
- [GitHub Repository of FunQG](https://github.com/hhaji/funqg)

In [1]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

#### Set Path

In [2]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/" 

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True) 

path_save = current_dir + "graph_data.zip"
shutil.unpack_archive(path_save, folder_data_temp)

#### Custom PyTorch Datasets

In [3]:
""" Classification Dataset """
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
            self.address=address+".bin"
            self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
            num_graphs =len(self.list_graphs)
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
    def __len__(self):
        return len(self.list_graphs)
    def __getitem__(self, idx):
        return  self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]

#### Defining Train, Validation, and Test Set

In [4]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))


1210 151 152


#### Data Loader

In [5]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [6]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64) 

#### Defining A GNN

##### Some Variables

In [7]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1 

# Size of global feature of each graph 
global_size = 200 

# Number of epochs to train the model
num_epochs = 100 

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10 

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100} 


In [8]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks
        
        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size] 
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function to Compute Score of the Model

In [9]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [10]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [11]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [12]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:    
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1        

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")        
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [13]:
def test_evaluate():  
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [14]:
import time
start_time = time.time()

train_evaluate()        
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.686 | Valid Score: 0.562
 
Epoch: 1/100 | Best Valid Score Until Now: 0.562 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.686 | Valid Score: 0.567
 
Epoch: 2/100 | Best Valid Score Until Now: 0.567 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.685 | Valid Score: 0.574
 
Epoch: 3/100 | Best Valid Score Until Now: 0.574 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.684 | Valid Score: 0.581
 
Epoch: 4/100 | Best Valid Score Until Now: 0.581 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.683 | Valid Score: 0.587
 
Epoch: 5/100 | Best Valid Score Until Now: 0.587 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.683 | Valid Score: 0.595
 
Epoch: 6/100 | Best Valid Score Until Now: 0.595 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.682 | Valid Score: 0.601
 
Epoch: 7/100 | Best Valid Score Until Now: 0.601 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.681 | Valid Score: 0.604
 
Epoch: 8/100 | Best Valid Score Until Now: 